In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn.parameter import Parameter
# import Transforms
%matplotlib inline
import pylab as pl
from IPython import display
import numpy as np
import time

In [2]:
data = torch.load("../mIN_train.pth")

In [3]:
torch.cuda.set_device(1)
a = 512
newdata = torch.stack([torch.ByteTensor(data[0][i]).float().view(84,84,3).transpose(0,2).contiguous() 
                       for i in range(64)],0)
newdata = Variable(newdata).cuda()
criterion = nn.MSELoss()
targ = Variable(torch.randn(64,a,16,16)).cuda()

In [11]:
from gpytorch.utils.fft import fft3,ifft3
from torch.autograd import Variable, Function
from torch.nn.parameter import Parameter

class FFT3(Function):
    def __init__(self):
        super(FFT3, self).__init__()
    
    def forward(self, inp):
        return fft3(inp)
    
    def backward(self, grad):
        return ifft3(grad)

class FFT3_up(Function):
    def __init__(self,bsize):
        super(FFT3_up, self).__init__()
        self.bsize = bsize
    
    def forward(self, inp):
        result = fft3(inp)
        return result.expand(self.bsize,inp.size(1),inp.size(2),inp.size(3)//2+1,2)
    
    def backward(self, grad):
        return ifft3(grad).sum(0).unsqueeze(0)
    
class iFFT3(Function):
    def __init__(self):
        super(iFFT3, self).__init__()
        
    def forward(self, inp):
        return ifft3(inp)
    
    def backward(self, grad):
        return fft3(grad)
    
class Conv3dF(nn.Module):
    def __init__(self,nchannels,size):
        super(Conv3dF,self).__init__()
        self.nchannels = nchannels
        self.size = size
        self.fft = FFT3()
        self.ifft = iFFT3()
        self.fftup = FFT3_up(64)
        self.padder = nn.ConstantPad2d((0,size-3,0,size-3),0)
        self.weights = Parameter(torch.randn(nchannels,3,3).view(1,nchannels,3,3)) 
                                                # TODO proper weight initialization
        
    def forward(self, inp):
        assert inp.size(2)==self.size and inp.size(3)==self.size
        fi = self.fft(inp) #self.fft(F.pad(inp,(1,1,1,1)))
        fw = self.fftup(self.padder(self.weights))
        result = torch.cat([(fi[:,:,:,:,0]*fw[:,:,:,:,0]-fi[:,:,:,:,1]*fw[:,:,:,:,1]).unsqueeze(4),
                            (fi[:,:,:,:,0]*fw[:,:,:,:,1]+fi[:,:,:,:,1]*fw[:,:,:,:,0]).unsqueeze(4)],dim=4)
#         k = fw[:,:,:,:,0]*fi.sum(4)
#         d = fi[:,:,:,:,1]*fw.sum(4)
#         c = fi[:,:,:,:,0]*(fw[:,:,:,:,0]-fw[:,:,:,:,1])
#         result = torch.cat([(k-d).unsqueeze(4),
#                             (k-c).unsqueeze(4)],dim=4)
        return self.ifft(result) #[:,:,1:-1,1:-1]
        

In [12]:
class testnet(nn.Module):
    def __init__(self):
        super(testnet, self).__init__()
        self.process = nn.Sequential(
            nn.MaxPool2d(5),
            nn.Conv2d(3,a,(1,1),padding=0),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU(),
            Conv3dF(a,16),
            nn.ReLU()
        )
        
    def forward(self, inp):
        return self.process(inp)

net = testnet().cuda()
nweights = sum([i.numel() for i in list(net.parameters())])
print(nweights," parameters in Fourier neural net.")

29696  parameters in Fourier neural net.


In [13]:
# FOURIER 3D
start = time.time()
for i in range(1):
    net.zero_grad()
    out = net(newdata)
    loss = criterion(out,targ)
    loss.backward()
print(time.time()-start)
# 3057 Mb  64
# 5563 Mb  128
# 10589 Mb  256

# 887 Mb  256 16
# 1263 Mb  512 16   (!!!) (Time-equivalent too!)

0.18209481239318848
